In [ ]:
# 1 & 2

import pandas as pd
from sqlalchemy import create_engine, text

def rentals_month(engine, month: int, year: int) -> pd.DataFrame:
    """
    Retrieves rental data for a given month and year from the Sakila database.

    Parameters:
        engine: SQLAlchemy engine object connected to the Sakila database.
        month (int): The month (1-12) for which rental data should be retrieved.
        year (int): The year for which rental data should be retrieved.

    Returns:
        pd.DataFrame: DataFrame containing the rental data for the given month/year.
    """
    
    query = text("""
        SELECT r.rental_id,
               r.rental_date,
               r.inventory_id,
               r.customer_id,
               r.return_date,
               r.staff_id
        FROM rental r
        WHERE EXTRACT(MONTH FROM r.rental_date) = :month
          AND EXTRACT(YEAR FROM r.rental_date) = :year
    """)
    
    with engine.connect() as conn:
        df = pd.read_sql(query, conn, params={"month": month, "year": year})
    
    return df


In [6]:

import pandas as pd
import pymysql
from sqlalchemy import create_engine, text

from getpass import getpass

In [8]:
pwd = getpass()
USER = "root"
HOST = "localhost"  
BD = "sakila"

In [9]:
con_url = f"mysql+pymysql://{USER}:{pwd}@{HOST}/{BD}"

engine = create_engine(con_url)

In [10]:
# 3

def rental_count_month(df: pd.DataFrame, month: int, year: int) -> pd.DataFrame:
    """
    Counts the number of rentals per customer for a given month and year.

    Parameters:
        df (pd.DataFrame): DataFrame returned by rentals_month().
        month (int): The month for which rentals are counted.
        year (int): The year for which rentals are counted.

    Returns:
        pd.DataFrame: DataFrame with customer_id and their rental count,
                      with the rental count column named 'rentals_MM_YYYY'.
    """

    # Format the month with leading zero (e.g., 05)
    month_str = f"{month:02d}"
    col_name = f"rentals_{month_str}_{year}"

    # Group by customer_id and count rentals
    rental_counts = (
        df.groupby("customer_id")
          .size()
          .reset_index(name=col_name)
    )

    return rental_counts


In [11]:
# Have an engine and imported rentals_month
# Rentals for May 2005
df_may = rentals_month(engine, 5, 2005)

# Count rentals per customer
rental_counts_may = rental_count_month(df_may, 5, 2005)
print(rental_counts_may.head())

   customer_id  rentals_05_2005
0            1                2
1            2                1
2            3                2
3            5                3
4            6                3


In [12]:
# 4 

def compare_rentals(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Compares rental counts between two months.

    Parameters:
        df1 (pd.DataFrame): DataFrame with rental counts for the first month (from rental_count_month).
        df2 (pd.DataFrame): DataFrame with rental counts for the second month (from rental_count_month).

    Returns:
        pd.DataFrame: Combined DataFrame with customer_id, rental counts for both months,
                      and a 'difference' column (df2 - df1).
    """
    
    # Merge on customer_id (inner join ensures only customers active in both months)
    merged = pd.merge(df1, df2, on="customer_id", how="inner")

    # Identify rental count columns (they are the second and third columns)
    col1 = merged.columns[1]
    col2 = merged.columns[2]

    # Add difference column
    merged["difference"] = merged[col2] - merged[col1]

    return merged


In [13]:
# Rentals for May and June 2005
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

# Counts
counts_may = rental_count_month(df_may, 5, 2005)
counts_june = rental_count_month(df_june, 6, 2005)

# Compare
comparison = compare_rentals(counts_may, counts_june)
print(comparison.head())

   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7           5
1            2                1                1           0
2            3                2                4           2
3            5                3                5           2
4            6                3                4           1
